In [1]:
import xarray as xr
import pandas as pd

In [2]:
#GOALS: 
#Correlation matrix between the ngpus and the norksy800
#Spectral analysis between the ngpus and the norkyst800
#plot f/h as contourlines with u & v
ds = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/experiments/ngpus-2017-24/inference/lam-48h-last.nc')

In [3]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:            (time: 17, values: 3076056)
Coordinates:
  * time               (time) datetime64[ns] 136B 2024-04-02 ... 2024-04-04
Dimensions without coordinates: values
Data variables: (12/27)
    latitude           (values) float32 12MB ...
    longitude          (values) float32 12MB ...
    h                  (time, values) float32 209MB ...
    salinity_0         (time, values) float32 209MB ...
    sea_mask           (time, values) float32 209MB ...
    temperature_0      (time, values) float32 209MB ...
    ...                 ...
    rain               (time, values) float32 209MB ...
    river_binary_mask  (time, values) float32 209MB ...
    sin_julian_day     (time, values) float32 209MB ...
    sin_latitude       (time, values) float32 209MB ...
    sin_local_time     (time, values) float32 209MB ...
    sin_longitude      (time, values) float32 209MB ...

In [5]:
import numpy as np

Spectral analysis: The method

$dct[i][j] = ci * cj (\sum(k=0 to m-1) \sum(l=0 to n-1) matrix[k][l] * cos((2*k+1) *i*\pi/2*m) * cos((2*l+1) *j*\pi/2*n)$ 
where $ci= 1/\sqrt{m}$ if i=0 else $ci= \sqrt{2}/\sqrt{m}$ and 
similarly, $cj= 1/\sqrt{n}$ if j=0 else $cj= \sqrt{2}/\sqrt{n}$ 

In [ ]:
m = 8
n = 8

def dc_transform(matrix):
    pi = np.pi
    cos = np.cos
    dct = []
    for i in range(m):
        dct.append([None]*2)
    
    for i in range(m):
        for j in range(n):
            
            if (i==0):
                beta_m = np.sqrt(1/(m))
            elif (i!=0):
                beta_m = np.sqrt(2/(m))
            
            if (j==0):
                beta_n = np.sqrt(1/(n))
            elif (j!=0):
                beta_n = np.sqrt(2/(n))

            sum = 0 
            for k in range(m):
                for l in range(n):

                    dct_sum = np.matrix[k][l] * (cos * ((i+(1/2)*pi*k)/m)) * (cos * ((j+(1/2)*pi*l)/n)) 
                    



